찬우: Output 데이터 양산

In [2]:
import json
import re
from error_dict import dictionary 

# 대체어 그룹 정의
def merge_dicts(*dicts):
    merged_dict = {}
    for dictionary in dicts:
        for key, value in dictionary.items():
            if key in merged_dict:
                print(key, value)
                merged_dict[key].extend(value)
            else:
                merged_dict[key] = value
    return merged_dict

replacement_groups = merge_dicts(dictionary)

# 정규 표현식 패턴 생성
replacement_patterns = {
    replacement: re.compile(r"\b" + r"\b|\b".join(re.escape(term) for term in terms) + r"\b", re.IGNORECASE)
    for replacement, terms in replacement_groups.items()
}

In [4]:
replacement_groups['산']

['mountain', 'mountains', 'peak', 'hill', 'summit']

In [6]:
# JSON 파일 읽기
for i in range(10084, 10085): # (각자 구축량 시작점, 각자 구축량 마감개수 + 1) 
    file_path = f'../front/public/json/outputJson/output_{i}.json' # 수정없이 걍 돌려도 됨
    
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)

    new_bounding_boxes = data.get('new_bounding_boxes', [])

    for box_index, box in enumerate(new_bounding_boxes):
        for caption_idx, caption in enumerate(box['captions']):
            for errorCaption_idx, errorCaption in enumerate(caption['errorCaption']):
                if caption['caption'].strip() == errorCaption.strip():
                    for replacement, pattern in replacement_patterns.items():
                        if pattern.search(errorCaption.lower()):
                            # 대체어로 변경
                            new_bounding_boxes[box_index]['captions'][caption_idx]['errorCaption'][errorCaption_idx] = pattern.sub(replacement, errorCaption.lower())
                            break

    data['new_bounding_boxes'] = new_bounding_boxes
    
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=4)

# 복구

In [8]:
# wrong_dict
wrong_dict = {
                'crape': ['ripples'],
    'young': ['japanese macaque'],
                'ripples': ['crape'],
                'cartoon': ['magazine', 'magazines'],
                'palm': ['fist', 'knuckles', 'punch'],
                'small bag': ['pocketbook', 'purse', 'wallet'],
                'beads': ['pearl', 'pearls'],
                'sawdust pile': ['bales'],
                'boat': ['ca', 'van', 'vehicle', 'vehicles'],
                'tent': ['awning', 'awnings'],
                'torch': ['candle', 'candles', 'second'],
                'hair': ['prong', 'prongs'],
                'meeting': ['conference'],
                'scissors': ['cutter', 'knife', 'knifes', 'knives'],
                'party list': ['minutes'],
                'warehouse': ['shop', 'shops', 'store'],
                'wooden fence': ['guardrail'],
                'jacket': ['cardigan'],
                'bar': ['karaoke'],
                'iron pole': ['spruce'],
                'plastic bag': ['bag', 'bags', 'satchel'],
                'audience': ['singer'],
                'firewood': ['burners', 'stove', 'burner'],
                'casual': ['gown', 'robe'],
                'blade': ['brim', 'brims', 'edge', 'edges', 'fringe', 'verge'],
                'friends': ['household', 'family'],
                'stick, wand': ['boughs', 'branch', 'branches', 'eggplant', 'sprig', 'tine'],
                'pet': ['livestock'],
                'meal': ['snack', 'snacks'],
                'sofa': ['cot'],
                'road sign': ['signboard'],
                'hanger': ['hooks'],
                'rice': ['reed', 'reeds'],
                'kite': ['gull', 'parachute', 'kite', 'kites'],
                'dog chew': ['ribs'],
                'yellow': ['brown', 'browns', 'sepia', 'tan'],
                'raven hair': ['brunet'],
                'fin': ['caudal'],
                'snack': ['meal', 'meals'],
                'smoke': ['mist'],
                'beret': ['hats'],
                'hallway': ['hall'],
                'circuit': ['parade'],
                'hood': ['top'],
                'wind turbine': ['substation'],
                'bonsai': ['topiary'],
                'wood': ['log', 'logs'],
                'mushroom': ['truffle'],
                'mossy stone': ['rock', 'rocks'],
                'calendar': ['plan', 'plans', 'schedule'],
                'bread': ['toast'],
                'fur': ['hair'],
                'building': ['stadium'],
                'plate': ['dish'],
                'wing': ['feather'],
    }

In [ ]:
start_num = 1095
end_num = 1459
right_words = list(wrong_dict.keys()) # error caption

# 잘못된 단어로 변경된 경우 전체 복구 함수
def repair_sentence(start_num, end_num, right_word, wrong_word):
    def find_differences_using_sets(sentence1, sentence2):
        # 문장을 단어 단위로 분리
        words1 = set(sentence1.split())
        words2 = set(sentence2.split())
        
        # 차집합을 이용해 서로 다른 단어를 찾음
        diff1 = words1 - words2
        diff2 = words2 - words1
        
        return diff1, diff2

    # JSON 파일 읽기
    for i in range(start_num, end_num): # (각자 구축량 시작점, 각자 구축량 마감개수 + 1) 
        file_path = f'../front/public/json/outputJson/output_{i}.json' # 수정없이 걍 돌려도 됨
        
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)

        new_bounding_boxes = data.get('new_bounding_boxes', [])
        print(i)
        for box_index, box in enumerate(new_bounding_boxes):
            for caption_idx, caption in enumerate(box['captions']):
                for errorCaption_idx, errorCaption in enumerate(caption['errorCaption']):
                    
                    if caption['caption'].strip() != errorCaption.strip():
                        if re.search(rf'\b{wrong_word}\b', errorCaption.lower()):
                            if re.search(rf'\b{right_word}\b', caption['caption'].strip().lower()):
                                # 대체어로 변경
                                origin = new_bounding_boxes[box_index]['captions'][caption_idx]['caption']
                                print(origin)
                                modified = new_bounding_boxes[box_index]['captions'][caption_idx]['errorCaption'][errorCaption_idx] 
                                print(modified)
                                new_bounding_boxes[box_index]['captions'][caption_idx]['errorCaption'][errorCaption_idx]  = origin#= pattern.sub(replacement, errorCaption.lower())
                                diffs1, diffs2 = find_differences_using_sets(origin, modified)
                                print(f"Sentence 1 has unique words: {diffs1}")
                                print(f"Sentence 2 has unique words: {diffs2}")


        data['new_bounding_boxes'] = new_bounding_boxes
        
        with open(file_path, 'w', encoding='utf-8') as f:
            json.dump(data, f, indent=4)


[[repair_sentence(start_num, end_num, right_word, wrong_word) for wrong_word in wrong_dict[right_word]] for right_word in right_words]